In [ ]:
import s3fs
import xarray as xr 
import numpy as np 
import matplotlib.pyplot as plt 
 

In [ ]:
fsg = s3fs.S3FileSystem(anon=False,
      client_kwargs={
         'endpoint_url': 'https://karen.uiogeo-apps.sigma2.no'
      })

In [ ]:
fgs.ls('data')

In [ ]:
data_path = 's3://data/*.nc'

In [ ]:
remote_files = fsg.glob(data_path)
remote_files

In [ ]:
fileset = [fsg.open(file) for file in remote_files]
#combine all datasets on coords
dset = xr.open_mfdataset(fileset, combine='by_coords')

In [ ]:
# files needed 
v = xr.open_mfdataset(filedir + 'vbar.nc')
u = xr.open_mfdataset(filedir + 'ubar.nc')
bat = xr.open_mfdataset(filedir + '/CREG12.L75.script/bathym.nc')
dyp = xr.open_mfdataset(filedir + 'depth.nc')
cmap_r = plt.get_cmap("magma_r")

In [ ]:
# Velocity data 
u = u*73
v = v*73
vel = (u.vozocrtx**2 + v.vomecrty**2)**0.5

In [ ]:
# Smoothening Velocity data
us = u.rolling(x=10).mean()
vs = v.rolling(y=10).mean()
vels = (us.vozocrtx**2 + vs.vomecrty**2)**0.5


In [ ]:
# Quiver

c = 15
N = 100

x = np.linspace(0, 1801, 1801)
dx = 20 
x.shape

y = np.linspace(0, 1580, 1580)
dy = 20
y.shape

X, Y = np.meshgrid(x, y)
print(X.shape)
print(us.vozocrtx.shape)
